In [7]:
import base64
import json
from pygltflib import GLTF2, BufferFormat
import struct
DATA_URI_HEADER = "data:application/octet-stream;base64,"

In [8]:
gltf = GLTF2.load('model_four.gltf') 

In [9]:
position_list           = []
normal_list             = []
texcoord_list           = []
triangle_indicies_list  = []
bone_weight_list        = []
bone_indicies_list      = []
# bone_rest_pos           = []
# bone_rest_rot           = []
for mesh in gltf.meshes:
    for primitive in mesh.primitives:
        position_list.append(primitive.attributes.POSITION)
        normal_list.append(primitive.attributes.NORMAL)
        texcoord_list.append(primitive.attributes.TEXCOORD_0)
        triangle_indicies_list.append(primitive.indices)
        
        bone_weight_list.append(primitive.attributes.WEIGHTS_0)
        bone_indicies_list.append(primitive.attributes.JOINTS_0)

all_lists = [position_list, normal_list, texcoord_list, triangle_indicies_list, bone_weight_list, bone_indicies_list]
all_lists


[[0, 7, 13, 19],
 [1, 8, 14, 20],
 [2, 9, 15, 21],
 [5, 12, 18, 25],
 [4, 11, 17, 24],
 [3, 10, 16, 23]]

In [16]:
with open("character_sofunny_model_four.bin", "wb") as f:
    ROWS = 4
    uri = gltf.buffers[0].uri
    all_data = uri.split(DATA_URI_HEADER)
    data = uri.split(DATA_URI_HEADER)[1]
    data = base64.b64decode(bytes(data,"utf8"))
    for attribute in all_lists:
        print(attribute)
        # get total count of current attribute
        # get data locations for current attribute
        total_count_of_attribute = 0
        to_write = []
        for accessor_index in attribute:
            accessor = gltf.accessors[accessor_index]
            total_count_of_attribute += accessor.count
            buffer_view = gltf.bufferViews[accessor.bufferView]
            byte_offset = buffer_view.byteOffset
            byte_length = buffer_view.byteLength
            to_write.append((byte_offset, byte_length))

        print(total_count_of_attribute)
        print(to_write)
        # write data to file

        # write total count of current attribute
        f.write(total_count_of_attribute.to_bytes(4, byteorder='little', signed=False))
        if(attribute[0] == 4 or attribute[0] == 3):
            f.write(ROWS.to_bytes(4, byteorder='little', signed=False))
        # write data locations for current attribute
        for byte_offset, byte_length in to_write:
            f.write(data[byte_offset:byte_offset+byte_length])
        
    tc = len(gltf.nodes)
    f.write(tc.to_bytes(4, byteorder='little', signed=False))
    print(tc)
    for node in gltf.nodes:
        if(node.translation is None):
            continue
        for val in node.translation:
            f.write(struct.pack('f', val))
    f.write(tc.to_bytes(4, byteorder='little', signed=False))
    print(tc)
    for node in gltf.nodes:
        if(node.translation is None):
            continue
        for val in node.rotation:
            f.write(struct.pack('f', val))

[0, 7, 13, 19]
27764
[(0, 108192), (10176932, 106068), (20101436, 48780), (20354368, 70128)]
[1, 8, 14, 20]
27764
[(108192, 108192), (10283000, 106068), (20150216, 48780), (20424496, 70128)]
[2, 9, 15, 21]
27764
[(216384, 72128), (10389068, 70712), (20198996, 32520), (20494624, 46752)]
[5, 12, 18, 25]
124710
[(468832, 84582), (10636560, 73158), (20312816, 41550), (20705008, 50130)]
[4, 11, 17, 24]
27764
[(324576, 144256), (10495136, 141424), (20247776, 65040), (20611504, 93504)]
[3, 10, 16, 23]
27764
[(288512, 36064), (10459780, 35356), (20231516, 16260), (20588128, 23376)]
71
71


In [127]:
x = 0
with open("character_sofunny_model_four.bin", "wb") as f: 
    uri = gltf.buffers[0].uri
    all_data = uri.split(DATA_URI_HEADER)
    data = uri.split(DATA_URI_HEADER)[1]
    data = base64.b64decode(bytes(data,"utf8"))
    total_list = []
    for attributes in all_lists:
        total_counter = 0
        
        #get data from uri

        #start and ending bytes of each list
        temp_list = []

        for temp_index in attributes:
            print("temp index:\t", str(temp_index))
            accessor_index = gltf.accessors[temp_index].bufferView
            print("accessor_index:\t", str(accessor_index))
            total_counter += gltf.accessors[temp_index].count
            print(gltf.accessors[temp_index].count, end=" ")
            print()
            byteOS = gltf.bufferViews[accessor_index].byteOffset
            bytelength = gltf.bufferViews[accessor_index].byteLength
            temp_list.append((byteOS, byteOS + bytelength, accessor_index))
        # print(total_counter)
        f.write(total_counter.to_bytes(4, byteorder='little', signed=False))
        if(x == 4 or x == 5):
            temp = 4
            f.write(temp.to_bytes(4, byteorder='little', signed=False))
        total_list.append(total_counter)
        for start, end, accessor_index in temp_list:
            x += 1
            print(str(x) +" " + gltf.accessors[temp_index].type)
            print('\t', end="")
            print(start, end)
            print('\t', end="")
            print(gltf.bufferViews[accessor_index].byteLength, end-start)
            print('\t', end="")
            f.write(data[start:end])

    tc = len(gltf.nodes)
    f.write(tc.to_bytes(4, byteorder='little', signed=False))
    for node in gltf.nodes:
        if(node.translation is None):
            continue
        for val in node.translation:
            f.write(struct.pack('f', val))
    f.write(tc.to_bytes(4, byteorder='little', signed=False))
    for node in gltf.nodes:
        if(node.translation is None):
            continue
        for val in node.rotation:
            f.write(struct.pack('f', val))
            

    # look through each list, add total count of the 

temp index:	 0
accessor_index:	 0
9016 
temp index:	 7
accessor_index:	 8
8839 
temp index:	 13
accessor_index:	 15
4065 
temp index:	 19
accessor_index:	 21
5844 
1 VEC3
	0 108192
	108192 108192
	2 VEC3
	10176932 10283000
	106068 106068
	3 VEC3
	20101436 20150216
	48780 48780
	4 VEC3
	20354368 20424496
	70128 70128
	temp index:	 1
accessor_index:	 1
9016 
temp index:	 8
accessor_index:	 9
8839 
temp index:	 14
accessor_index:	 16
4065 
temp index:	 20
accessor_index:	 22
5844 
5 VEC3
	108192 216384
	108192 108192
	6 VEC3
	10283000 10389068
	106068 106068
	7 VEC3
	20150216 20198996
	48780 48780
	8 VEC3
	20424496 20494624
	70128 70128
	temp index:	 2
accessor_index:	 2
9016 
temp index:	 9
accessor_index:	 10
8839 
temp index:	 15
accessor_index:	 17
4065 
temp index:	 21
accessor_index:	 23
5844 
9 VEC2
	216384 288512
	72128 72128
	10 VEC2
	10389068 10459780
	70712 70712
	11 VEC2
	20198996 20231516
	32520 32520
	12 VEC2
	20494624 20541376
	46752 46752
	temp index:	 5
accessor_index:	 5

[-1.3170389223660095e-08, -0.9899873733520508, 1.8814839464198485e-08, 0.14115624129772186]


In [104]:
total_list

[27764, 27764, 27764, 124710, 27764, 27764]

In [105]:
gltf.accessors[7].bufferView

8

In [106]:
x = 468832    
x = 10636624
x = 20312880
x = 20705072
x = x + 2 * 10000
for i in range(100):
    y = struct.unpack("H", data[x:x+2])
    x += 2
    print(y)

(2413,)
(2412,)
(2337,)
(2413,)
(2337,)
(2326,)
(2409,)
(2413,)
(2326,)
(2409,)
(2326,)
(2349,)
(2403,)
(2402,)
(2414,)
(2403,)
(2414,)
(2407,)
(2409,)
(2408,)
(2415,)
(2409,)
(2415,)
(2416,)
(2417,)
(2415,)
(2408,)
(2417,)
(2408,)
(2407,)
(2418,)
(2417,)
(2407,)
(2418,)
(2407,)
(2414,)
(2416,)
(2413,)
(2409,)
(2412,)
(2413,)
(2416,)
(2412,)
(2416,)
(2415,)
(2410,)
(2412,)
(2415,)
(2410,)
(2415,)
(2417,)
(2411,)
(2410,)
(2417,)
(2411,)
(2417,)
(2418,)
(2420,)
(2418,)
(2414,)
(2420,)
(2414,)
(2419,)
(2419,)
(2406,)
(2422,)
(2419,)
(2422,)
(2423,)
(2339,)
(2420,)
(2419,)
(2339,)
(2419,)
(2423,)
(2411,)
(2418,)
(2420,)
(2411,)
(2420,)
(2339,)
(2423,)
(2336,)
(2339,)
(2334,)
(2336,)
(2423,)
(2334,)
(2423,)
(2424,)
(2426,)
(2424,)
(2423,)
(2426,)
(2423,)
(2422,)
(2427,)
(2335,)
(2334,)
(2427,)


In [86]:
with open('character.bin', 'rb') as f:
    print(int.from_bytes(f.read(4), byteorder='little', signed=False))
    


5002


In [107]:
# test

with open("character.bin", "rb") as f:
    types = ['vec3', 'vec3', 'vec2', 'unsigned short', '2dfloat', '2dus', 'vec3', 'quat']
    for type in types:
        count = int.from_bytes(f.read(4), byteorder='little', signed=False)
        print(count)
        match type:
            case 'vec3':
                for i in range(count):
                    x = struct.unpack("HHH", f.read(6))
                    print(x)
                    break
        

5002
(38420, 48620, 14762)
4082646948
(48651, 20943, 48640)
1067889139
3188521455
3186731974
1068001446
3189039142
(63908, 48657, 6848)
1000619948
